In [176]:
import pandas as pd
import geopandas as gdp
import matplotlib.pyplot as plt
import numpy as np 

import re


#clean2gether = gdp.read_file('C:/Users/lilia/OneDrive/Documents/Lilian/ING4_S2/Untitled Folder/clean2gether.shp/clean2gether.shp')
#france=gdp.read_file('C:/Users/lilia/OneDrive/Documents/Lilian/ING4_S2/Untitled Folder/france')

data=pd.read_json('C:/Users/lilia/OneDrive/Documents/Lilian/ING4_S2/Untitled Folder/Clean2gether.json')

data


,latitude,longitude,town,postcode,epci,county,images
0,45.675400,0.242730,Ruelle-sur-Touvre,16600.0,CA du Grand Angoulême,Charente,[https://api.v2.clean2gether.com/uploads/5fe30...
1,45.590920,0.272790,Dirac,16410.0,CA du Grand Angoulême,Charente,[https://api.v2.clean2gether.com/uploads/0291_...
2,45.548250,0.122070,Mouthiers-sur-Boëme,16440.0,CA du Grand Angoulême,Charente,[https://api.v2.clean2gether.com/uploads/ca6a7...
3,45.644590,0.118690,Angoulême,16000.0,CA du Grand Angoulême,Charente,[https://api.v2.clean2gether.com/uploads/1_D0_...
4,45.703020,-0.304430,Boutiers-Saint-Trojan,16100.0,CA du Grand Cognac,Charente,[https://api.v2.clean2gether.com/uploads/E23_F...
...,...,...,...,...,...,...,...
745,45.700577,-0.302963,Boutiers-Saint-Trojan,16100.0,CA du Grand Cognac,Charente,[https://api.v2.clean2gether.com/uploads/c2g_u...
746,46.494843,0.296437,Iteuil,86240.0,CC des Vallées du Clain,Vienne,[https://api.v2.clean2gether.com/uploads/c2g_u...
747,45.700729,-0.303293,Boutiers-Saint-Trojan,16100.0,CA du Grand Cognac,Charente,[https://api.v2.clean2gether.com/uploads/c2g_u...
748,43.418009,-1.473681,Ustaritz,64480.0,CA du Pays Basque,Pyrénées-Atlantiques,[https://api.v2.clean2gether.com/uploads/c2g_u...


In [75]:
def create_data(data):

    df=data[:]

    county=df["county"]
    town=df["town"]
    epci=df["epci"]

    county = pd.DataFrame(df.pivot_table(index = ['county'], aggfunc ='size') , columns=["nb"])
    town = pd.DataFrame(df.pivot_table(index = ['town'], aggfunc ='size') , columns=["nb"])
    epci = pd.DataFrame(df.pivot_table(index = ['epci'], aggfunc ='size'), columns=["nb"])  

    county["num_county"]=np.arange(1,len(county)+1)
    town["num_town"]=np.arange(1,len(town)+1)
    epci["num_epci"]=np.arange(1,len(epci)+1)

    county["weight"]=county["nb"]/len(county)
    town["weight"]=town["nb"]/len(town)
    epci["weight"]=epci["nb"]/len(epci)

    county["county"]=county.index
    town["town"]=town.index
    epci["epci"]=epci.index

    df["weight"]=np.zeros(len(df))

    for index, row in df.iterrows():
        for index2, row2 in county.iterrows():
            if row["county"]==row2["county"]:
                df["weight"][index]=row["weight"]+row2["weight"]
                df["county"][index]=row2["num_county"]

    for index, row in df.iterrows():
        for index2, row2 in town.iterrows():
            if row["town"]==row2["town"]:
                df["weight"][index]=row["weight"]+row2["weight"]
                df["town"][index]=row2["num_town"]
    
    for index, row in df.iterrows():
        for index2, row2 in epci.iterrows():
            if row["epci"]==row2["epci"]:
                df["weight"][index]=row["weight"]+row2["weight"]
                df["epci"][index]=row2["num_epci"]

    del df['images']
    
    return df
            
    

In [76]:
df=create_data(data)

df.to_csv (r'C:\Users\lilia\OneDrive\Documents\Lilian\ING4_S2\Untitled Folder\regression.csv', index = False, header=True)

<ipython-input-75-e73acff2d330>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight"]=np.zeros(len(df))
<ipython-input-75-e73acff2d330>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weight"][index]=row["weight"]+row2["weight"]
<ipython-input-76-e65d94580cb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df=create_data(data)
<ipython-input-75-e73acff2d330>:31: 

In [164]:
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data=pd.read_csv(r'C:\Users\lilia\OneDrive\Documents\Lilian\ING4_S2\Untitled Folder\regression.csv')

data=data.dropna()

x = data[["weight","postcode","epci","town","county"]]
y = data["longitude"]

x_train, x_test, y_trainlong, y_testlong = train_test_split(x, y, test_size = 0.15, random_state = None)

mlrlong = LinearRegression()  
mlrlong.fit(x_train, y_trainlong)

y_pred_mlrlong=mlrlong.predict(x_test)

r2 = r2_score(y_testlong, y_pred_mlrlong)
rmse = mean_squared_error(y_testlong, y_pred_mlrlong, squared=False)
print(r2)
print(rmse)


0.07255660376084061
0.5723561832441931


In [165]:
x = data[["weight","postcode","epci","town","county"]]
y = data["latitude"]

x_train, x_test, y_trainlat, y_testlat = train_test_split(x, y, test_size = 0.15, random_state = None)

mlrlat = LinearRegression()  
mlrlat.fit(x_train, y_trainlat)

y_pred_mlrlat=mlrlat.predict(x_test)

r2 = r2_score(y_testlat, y_pred_mlrlat)
rmse = mean_squared_error(y_testlat, y_pred_mlrlat, squared=False)
print(r2)
print(rmse)

0.6356801894842337
0.36140082863816037


In [173]:
y_pred_mlrlat=list(y_pred_mlrlat)
y_pred_mlrlong=list(y_pred_mlrlong)

prediction=pd.DataFrame()
prediction["longitude"]=y_pred_mlrlong
prediction["latitude"]=y_pred_mlrlat

train=pd.DataFrame()
train["latitude"]=y_trainlat
train["longitude"]=y_trainlong

train=train.dropna()


In [171]:
import folium

c = folium.Map(location=[ 44.833328, -0.56667])
#c.save('maCarte1.html')

for i in range(0,len(prediction)):
   folium.Circle(
      location=[prediction.iloc[i]['latitude'], prediction.iloc[i]['longitude']],
      radius=10,
      color="#FA8072",
      fill=False,
   ).add_to(c)

for i in range(0,len(train)):
   folium.Circle(
      location=[train.iloc[i]['latitude'], train.iloc[i]['longitude']],
      radius=10,
      color="#3186cc",
      fill=False,
      
   ).add_to(c)

In [172]:
c